In [1]:
# train a generative adversarial network on a one-dimensional function
from numpy import hstack
from numpy import zeros
from numpy import ones
from numpy.random import rand
from numpy.random import randn
from keras.models import Sequential
from keras.layers import Dense
from matplotlib import pyplot
import requests
import numpy as np
import pandas as pd

from bs4 import BeautifulSoup


In [2]:
file1 = open('cards-standard.txt', 'r') 
Lines = file1.readlines() 
maximum = len(Lines)
cards_key = {}
cards_value = {}
i=1
for line in Lines:
    cards_value[i] = line.rstrip("\n")
    cards_key[line.rstrip("\n")] = i
    i += 1

print(cards_key["Arcane Breath"])
print(cards_value[32])
print(len(Lines))

32
Arcane Breath
1254


In [3]:
classes_id = {'Demon Hunter' : 16384,
          'Druid' : 4,
          'Hunter' : 8,
          'Mage' : 16,
          'Paladin' : 32,
          'Priest' : 64, 
          'Rogue' : 128,
          'Shaman' : 256,
          'Warlock' : 512,
          'Warrior' : 1024}

r = requests.get('https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&filter-class='+str(classes_id['Demon Hunter']))
soup = BeautifulSoup(r.content,"html.parser")
rows = soup.find('ul', class_="b-pagination-list paging-list j-tablesorter-pager j-listing-pagination")
links = rows.find_all('li')
print(links[len(links)-2].text)


56


In [10]:
#scrape deck data from Hearthpwn website, standard decks

def page_max(deck_type):
    r = requests.get('https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&filter-class='+str(classes_id[deck_type]))
    soup = BeautifulSoup(r.content,"html.parser")
    rows = soup.find('ul', class_="b-pagination-list paging-list j-tablesorter-pager j-listing-pagination")
    links = rows.find_all('li')
    return int(links[len(links)-2].text)

def scrape_decks(deck_type):
    start_urls = [] 
    n = page_max(deck_type)
    print(n)
    for i in range(n):
        start_urls.append("https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=" + str(i+1)+"&filter-class="+ str(classes_id[deck_type]))
    #print(start_urls)
    deck_urls = []
    deck_classes = []
    for url in start_urls:
        print(url)
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        #rows = soup.find(id="decks-container")
 
        rows = soup.find_all('table', class_="listing listing-decks b-table b-table-a")
        for row in rows:
            decks = row.find_all('span', class_="tip")
            for deck in decks:
                links = deck.find_all('a', href=True)
                for link in links:
                    deck_urls.append("https://www.hearthpwn.com" + link['href'])
                    
            deck_class = row.find_all('td', class_="col-class")
            for cl in deck_class:
                deck_classes.append(cl.text)

    return cards_from_url(deck_urls), deck_classes



In [5]:
#scrape cards in deck from separate url
def cards_from_url(deck_urls):
    decks = []
    for url in deck_urls:
        print(url)
        deck=[]
        r = requests.get(url)
        soup = BeautifulSoup(r.content,"html.parser")
        table = soup.find_all('table', class_="listing listing-cards-tabular b-table b-table-a")
        for rows in table:
            row = rows.find_all('td', class_="col-name")
            for i in row:
                card = i.find('a')
                name = card.text
                number = card['data-count']

                for num in range(int(number)):
                    deck.append(name.replace('\n',''))
        #print(deck)
        #print(len(deck))
        decks.append(deck)
    return decks


        


In [6]:
def deck_to_vector(deck):
    deck_vector = []
    for elem in deck:
        #remove any potential white spaces from strings
        #if type(deck) is not list:
        elem = elem.lstrip()
        elem = elem.rstrip()

        deck_vector.append(cards_key[elem])
    return deck_vector

def vector_to_deck(deck_vector):
    deck = []
    for elem in deck_vector:
        deck.append(cards_value[elem])
    return deck

def vectorize_deck_list(decks):
    vecks = []
    for d in decks:
        vecks.append(deck_to_vector(d))
    return vecks

def concatenate_deck_class(decks, classes):
    concatenated_result = []
    for i in range(len(decks)):
        concatenated_result.append([decks[i],classes[i]])
    return concatenated_result


In [13]:
deck_type = ['Demon Hunter','Druid','Hunter','Mage','Paladin','Priest','Rogue','Shaman','Warlock','Warrior']
#deck_type = ['Hunter','Mage','Paladin','Priest','Rogue','Shaman','Warlock']
#deck_type = ['Paladin','Priest','Rogue','Shaman','Warlock']

for cl in deck_type:
    print("Scraping information: ", cl)
    decks, deck_classes = scrape_decks(cl)
    print("Creating ", cl ," decks")
    collected_decks = concatenate_deck_class(decks, deck_classes)
    collected_decks_vectored = concatenate_deck_class(vectorize_deck_list(decks), deck_classes)
    print("Creating ", cl, ".csv")
    #pd.DataFrame(collected_decks, columns=['cards', 'class']).to_csv("collected_decks_"+deck_type+".csv",index=False)
    pd.DataFrame(collected_decks_vectored, columns=['cards', 'class']).to_csv("collected_decks_vectored_"+cl+".csv",index=False)



Scraping information:  Paladin
73
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=1&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=2&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=3&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=4&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=5&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=6&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=7&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructe

https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=65&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=66&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=67&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=68&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=69&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=70&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=71&filter-class=32
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&

https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=56&filter-class=64
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=57&filter-class=64
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=58&filter-class=64
Creating  Priest  decks
Creating  Priest .csv
Scraping information:  Rogue
68
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=1&filter-class=128
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=2&filter-class=128
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=3&filter-class=128
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=4&filter-class=128
https://www.hearthpwn.com/

https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=61&filter-class=128
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=62&filter-class=128
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=63&filter-class=128
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=64&filter-class=128
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=65&filter-class=128
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=66&filter-class=128
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=67&filter-class=128
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck

https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=56&filter-class=256
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=57&filter-class=256
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=58&filter-class=256
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=59&filter-class=256
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=60&filter-class=256
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=61&filter-class=256
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=62&filter-class=256
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck

https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=50&filter-class=512
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=51&filter-class=512
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=52&filter-class=512
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=53&filter-class=512
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=54&filter-class=512
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=55&filter-class=512
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck-tag=1&page=56&filter-class=512
https://www.hearthpwn.com/decks?filter-show-standard=1&filter-show-constructed-only=y&filter-deck

In [ ]:
collected_decks = concatenate_deck_class(decks, deck_classes)
collected_decks_vectored = concatenate_deck_class(vectorize_deck_list(decks), deck_classes)


pd.DataFrame(collected_decks, columns=['cards', 'class']).to_csv("collected_decks_"+deck_type+".csv",index=False)
pd.DataFrame(collected_decks_vectored, columns=['cards', 'class']).to_csv("collected_decks_vectored_"+deck_type+".csv",index=False)

